# Single Cell RNA-seq Analysis Pipeline
A practical guide for analyzing scRNA-seq data following best practices.
This notebook walks through a complete single-cell analysis from raw counts to final cell type identification and visualization. 
Paper reference: PMC10832111

In [1]:
# =============================================================================
# Setup and Imports
# =============================================================================

import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scrublet as scr
import warnings
warnings.filterwarnings('ignore')
mpl.use('Agg')  # Use non-interactive backend


# Configure scanpy settings for better plots and analysis
sc.settings.verbosity = 3  # Set to 0 for less output, 3 for detailed logs
sc.settings.set_figure_params(dpi=100, facecolor='white', frameon=False)
#sc.settings.set_figure_params(dpi=80, facecolor='white', figsize=(6, 6))
sc.settings.n_jobs = -1  # Use all available cores

print("All packages loaded successfully!")
print(f"Scanpy version: {sc.__version__}")

All packages loaded successfully!
Scanpy version: 1.11.2


In [2]:
# =============================================================================
# Step 1: Load Your Data
# =============================================================================

"""
Data Loading Tips:
- For 10X data: use sc.read_10x_mtx() 
- For h5ad files: use sc.read_h5ad()
- For CSV files: use sc.read_csv()
- For h5 files: use sc.read_10x_h5()

The paper analyzed GSE194247 with 5 samples, focusing on epithelial and 
fibroblast populations from tissue samples.
"""

# Load your data - modify this path to match your file
data_path = "./data_and_results/scRNA_seq/GSE194247/adata_all_raw.h5ad"  # Change this to your file path
adata = sc.read_h5ad(data_path)

# Alternative loading methods (uncomment the one you need):
# adata = sc.read_10x_mtx('path/to/matrix/')  # For 10X mtx files
# adata = sc.read_10x_h5('path/to/file.h5')   # For 10X h5 files
# adata = sc.read_csv('path/to/file.csv')     # For CSV files

print(f"Loaded data: {adata.n_obs} cells × {adata.n_vars} genes")
print(f"Data type: {type(adata.X)}")

# Quick peek at the data structure
print("\nData overview:")
print(adata)

# Look at what metadata we have
print(f"\nCell metadata columns: {list(adata.obs.columns)}")
print(f"Gene metadata columns: {list(adata.var.columns)}")

# Check gene naming convention to help with MT gene detection
print(f"\nFirst 10 gene names: {adata.var_names[:10].tolist()}")
print(f"Last 10 gene names: {adata.var_names[-10:].tolist()}")

# Look for common mitochondrial gene patterns
mt_patterns = ['MT-', 'mt-', 'MITO', 'MT1', 'MT2']
for pattern in mt_patterns:
    count = sum(adata.var_names.str.contains(pattern, case=False))
    if count > 0:
        print(f"Found {count} genes containing '{pattern}'")

Loaded data: 51140 cells × 33538 genes
Data type: <class 'scipy.sparse._csr.csr_matrix'>

Data overview:
AnnData object with n_obs × n_vars = 51140 × 33538
    obs: 'sample_id', 'sample'
    var: 'gene_ids', 'feature_types'

Cell metadata columns: ['sample_id', 'sample']
Gene metadata columns: ['gene_ids', 'feature_types']

First 10 gene names: ['MIR1302-2HG', 'FAM138A', 'OR4F5', 'AL627309.1', 'AL627309.3', 'AL627309.2', 'AL627309.4', 'AL732372.1', 'OR4F29', 'AC114498.1']
Last 10 gene names: ['AC007325.2', 'BX072566.1', 'AL354822.1', 'AC023491.2', 'AC004556.1', 'AC233755.2', 'AC233755.1', 'AC240274.1', 'AC213203.1', 'FAM231C']
Found 14 genes containing 'MT-'
Found 14 genes containing 'mt-'
Found 40 genes containing 'MT1'
Found 22 genes containing 'MT2'


BIOLOGICAL OBSERVATION:
- In tissue samples like those in the paper, we expect heterogeneous cell populations
- The paper focused on epithelial cells (form barriers, involved in secretion/absorption)
- And fibroblast cells (provide structural support, ECM production)
- Raw data typically contains 15k-30k genes, but only ~2k-8k are expressed per cell
- Cell numbers vary by tissue type and dissociation efficiency
- Gene naming conventions vary between datasets - MT genes might be named differently

In [3]:
# =============================================================================
# Step 2: Basic Data Exploration and QC Metrics
# =============================================================================

"""
Quality Control (QC) is crucial in single-cell analysis. We calculate:
1. Number of genes expressed per cell (complexity indicator)
2. Total counts (UMI) per cell (sequencing depth)
3. Percentage of mitochondrial genes (indicates cell stress/death)
4. Percentage of ribosomal genes (metabolic activity indicator)

BIOLOGICAL CONTEXT from the paper:
- Healthy epithelial cells typically express 2000-6000 genes
- Fibroblasts often have higher gene expression diversity
- High MT% (>20%) usually indicates dying/stressed cells
- The paper used strict QC to ensure high-quality cell populations
"""

# Make gene names unique (important for downstream analysis)
adata.var_names_make_unique()

# Identify mitochondrial genes - try different naming conventions
adata.var['mt'] = (
    adata.var_names.str.startswith('MT-') |  # Human
    adata.var_names.str.startswith('mt-') |  # Mouse lowercase
    adata.var_names.str.startswith('MITO') | # Alternative naming
    adata.var_names.str.contains('^MT[0-9]') # Some datasets use MT1, MT2, etc.
)

# Check if we found any mitochondrial genes
n_mt_genes = adata.var['mt'].sum()
print(f"Found {n_mt_genes} mitochondrial genes")

if n_mt_genes > 0:
    # Show some examples of detected MT genes
    mt_genes = adata.var_names[adata.var['mt']].tolist()[:10]
    print(f"Example MT genes: {mt_genes}")
else:
    print("WARNING: No mitochondrial genes detected!")
    print("This might be normal for some datasets or if genes are named differently")

# Identify ribosomal genes (optional but useful)
adata.var['ribo'] = adata.var_names.str.startswith(('RPS', 'RPL'))

# Calculate QC metrics - this adds several columns to adata.obs
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)

# Check what QC columns were created
print(f"QC columns created: {[col for col in adata.obs.columns if 'counts' in col or 'genes' in col]}")

# Handle mitochondrial percentage - create it manually if not created
if 'pct_counts_mt' not in adata.obs.columns:
    if n_mt_genes > 0:
        # Calculate manually
        mt_gene_names = adata.var_names[adata.var['mt']]
        adata.obs['pct_counts_mt'] = (
            adata[:, mt_gene_names].X.sum(axis=1).A1 / adata.obs['total_counts'] * 100
        )
        print("Calculated pct_counts_mt manually")
    else:
        # Create dummy column with zeros if no MT genes found
        adata.obs['pct_counts_mt'] = 0.0
        print("Created dummy pct_counts_mt column (no MT genes found)")

# The calculate_qc_metrics function adds these useful columns:
# - total_counts: total UMI per cell
# - n_genes_by_counts: number of genes with >0 counts per cell  
# - pct_counts_mt: percentage of counts from mitochondrial genes (now guaranteed to exist)

print("QC metrics calculated!")
print(f"Mean genes per cell: {adata.obs['n_genes_by_counts'].mean():.0f}")
print(f"Mean UMI per cell: {adata.obs['total_counts'].mean():.0f}")
print(f"Mean MT%: {adata.obs['pct_counts_mt'].mean():.1f}%")

# Additional QC information
print(f"MT% range: {adata.obs['pct_counts_mt'].min():.1f}% - {adata.obs['pct_counts_mt'].max():.1f}%")
if adata.obs['pct_counts_mt'].max() == 0:
    print("NOTE: MT% is 0 for all cells - this suggests no mitochondrial genes were detected")
    print("This is normal for some processed datasets where MT genes were already filtered out")

Found 25 mitochondrial genes
Example MT genes: ['MT1HL1', 'MT4', 'MT3', 'MT2A', 'MT1E', 'MT1M', 'MT1A', 'MT1B', 'MT1F', 'MT1G']
QC columns created: ['n_genes_by_counts', 'total_counts']
Calculated pct_counts_mt manually
QC metrics calculated!
Mean genes per cell: 2171
Mean UMI per cell: 8396
Mean MT%: 14.2%
MT% range: 0.0% - 98.1%


BIOLOGICAL INTERPRETATION of QC metrics:
- Genes per cell: Healthy cells typically express 1000-7000 genes
  * Too few (<500): likely empty droplets or dead cells
  * Too many (>8000): potential doublets or highly active cells
- UMI counts: Reflects both cell size and transcriptional activity
  * Epithelial cells: often 2000-10000 UMIs
  * Fibroblasts: can have higher counts due to ECM gene expression
- MT percentage: Critical quality indicator (if MT genes are detected)
  * <5%: excellent quality cells
  * 5-10%: good quality (used in paper)
  * >15%: likely stressed/dying cells
  * NOTE: Some processed datasets may have MT genes pre-filtered

TROUBLESHOOTING: If no MT genes are detected, this could mean:
1. Different gene naming convention (check gene names above)
2. MT genes were already filtered out during preprocessing
3. Dataset uses non-standard identifiers
This is common in processed datasets and doesn't prevent analysis.

In [4]:
# =============================================================================
# Step 3: Doublet Detection with Scrublet
# =============================================================================

"""
Doublets are when two cells are captured in the same droplet, appearing as 
one cell with higher gene/UMI counts. This is particularly important in 
tissue studies where cell dissociation can create artificial associations.

BIOLOGICAL IMPACT:
- Doublets can create false "intermediate" cell states
- Particularly problematic for developmental trajectories
- The paper used Scrublet v0.2.2 to ensure singlet identification
- Expected doublet rates: ~6% for 10X protocols
"""

print("Running Scrublet doublet detection...")

# Initialize Scrublet - adjust expected_doublet_rate based on your protocol
# Typical rates: 10X v2 (~1% per 1000 cells), 10X v3 (~0.8% per 1000 cells)
scrub = scr.Scrublet(adata.X, expected_doublet_rate=0.06)

# Run doublet detection
doublet_scores, predicted_doublets = scrub.scrub_doublets(
    min_counts=2,           # Filter cells with < 2 UMIs
    min_cells=3,            # Filter genes in < 3 cells
    min_gene_variability_pctl=85,  # Keep top 15% most variable genes
    n_prin_comps=30         # PCs for doublet detection
)

# Add results to our data
adata.obs['doublet_score'] = doublet_scores
adata.obs['predicted_doublet'] = predicted_doublets

doublet_rate = predicted_doublets.mean() * 100
print(f"Predicted {predicted_doublets.sum()} doublets ({doublet_rate:.1f}% of cells)")

Running Scrublet doublet detection...
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.82
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.5%
Overall doublet rate:
	Expected   = 6.0%
	Estimated  = 0.0%
Elapsed time: 172.9 seconds
Predicted 0 doublets (0.0% of cells)


BIOLOGICAL OBSERVATION:
- Doublet rates >10% suggest technical issues in sample preparation
- In tissue samples, some apparent doublets might be real cell-cell interactions
- However, for clustering analysis, it's safer to remove predicted doublets
- The paper's stringent QC ensures clean cell type identification

In [5]:
# =============================================================================
# Step 4: Visualize QC Metrics Before Filtering
# =============================================================================

"""
Always visualize your QC metrics before filtering to understand your data
and choose appropriate thresholds.

WHAT TO LOOK FOR BIOLOGICALLY:
- Bimodal distributions often indicate multiple cell types or quality states
- Long tails in UMI/gene distributions suggest outliers
- MT% distributions reveal cell health status
"""

# Create QC plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Quality Control Metrics - Before Filtering', fontsize=16)

# Plot 1: Number of genes per cell
axes[0, 0].hist(adata.obs['n_genes_by_counts'], bins=50, alpha=0.7, color='skyblue')
axes[0, 0].axvline(500, color='red', linestyle='--', alpha=0.8, label='Min threshold (500)')
axes[0, 0].axvline(7000, color='red', linestyle='--', alpha=0.8, label='Max threshold (7000)')
axes[0, 0].set_xlabel('Number of genes')
axes[0, 0].set_ylabel('Number of cells')
axes[0, 0].set_title('Genes per cell')
axes[0, 0].legend()

# Plot 2: Total UMI counts per cell
axes[0, 1].hist(adata.obs['total_counts'], bins=50, alpha=0.7, color='lightgreen')
axes[0, 1].axvline(2000, color='red', linestyle='--', alpha=0.8, label='Min threshold (2000)')
axes[0, 1].set_xlabel('Total UMI counts')
axes[0, 1].set_ylabel('Number of cells')
axes[0, 1].set_title('UMI counts per cell')
axes[0, 1].legend()

# Plot 3: Mitochondrial gene percentage
if adata.obs['pct_counts_mt'].max() > 0:
    axes[0, 2].hist(adata.obs['pct_counts_mt'], bins=50, alpha=0.7, color='salmon')
    axes[0, 2].axvline(10, color='red', linestyle='--', alpha=0.8, label='Max threshold (10%)')
    axes[0, 2].set_xlabel('Mitochondrial gene %')
    axes[0, 2].set_ylabel('Number of cells')
    axes[0, 2].set_title('MT gene % per cell')
    axes[0, 2].legend()
else:
    axes[0, 2].text(0.5, 0.5, 'No MT genes\ndetected', ha='center', va='center', 
                    transform=axes[0, 2].transAxes, fontsize=12)
    axes[0, 2].set_title('MT gene % per cell')
    axes[0, 2].set_xlabel('Mitochondrial gene %')
    axes[0, 2].set_ylabel('Number of cells')

# Plot 4: Doublet scores
axes[1, 0].hist(adata.obs['doublet_score'], bins=50, alpha=0.7, color='plum')
axes[1, 0].set_xlabel('Doublet score')
axes[1, 0].set_ylabel('Number of cells')
axes[1, 0].set_title('Doublet scores')

# Plot 5: Genes vs UMI (colored by MT%)
if adata.obs['pct_counts_mt'].max() > 0:
    scatter = axes[1, 1].scatter(adata.obs['total_counts'], adata.obs['n_genes_by_counts'],
                                c=adata.obs['pct_counts_mt'], s=1, alpha=0.6, cmap='viridis')
    axes[1, 1].set_xlabel('Total UMI counts')
    axes[1, 1].set_ylabel('Number of genes')
    axes[1, 1].set_title('Genes vs UMI (colored by MT%)')
    plt.colorbar(scatter, ax=axes[1, 1])
else:
    axes[1, 1].scatter(adata.obs['total_counts'], adata.obs['n_genes_by_counts'],
                      s=1, alpha=0.6, color='blue')
    axes[1, 1].set_xlabel('Total UMI counts')
    axes[1, 1].set_ylabel('Number of genes')
    axes[1, 1].set_title('Genes vs UMI (no MT data)')

# Plot 6: UMI vs MT%
axes[1, 2].scatter(adata.obs['total_counts'], adata.obs['pct_counts_mt'], 
                   s=1, alpha=0.6, color='orange')
axes[1, 2].set_xlabel('Total UMI counts')
axes[1, 2].set_ylabel('Mitochondrial gene %')
axes[1, 2].set_title('UMI vs MT%')

plt.tight_layout()
plt.show()
plt.savefig('./figures/scrnaseq_qc_metrics_before_filtering.pdf', bbox_inches='tight')
plt.close()
# Print some useful statistics to help decide filtering thresholds
print("\nQC Statistics (useful for choosing thresholds):")
print(f"UMI counts - Min: {adata.obs['total_counts'].min()}, Max: {adata.obs['total_counts'].max()}")
print(f"UMI counts - 5th percentile: {adata.obs['total_counts'].quantile(0.05):.0f}")
print(f"UMI counts - 95th percentile: {adata.obs['total_counts'].quantile(0.95):.0f}")
print(f"Genes - Min: {adata.obs['n_genes_by_counts'].min()}, Max: {adata.obs['n_genes_by_counts'].max()}")
print(f"MT% - 95th percentile: {adata.obs['pct_counts_mt'].quantile(0.95):.1f}%")


QC Statistics (useful for choosing thresholds):
UMI counts - Min: 500.0, Max: 149695.0
UMI counts - 5th percentile: 664
UMI counts - 95th percentile: 25214
Genes - Min: 28, Max: 9225
MT% - 95th percentile: 72.5%


BIOLOGICAL INTERPRETATION OF QC PLOTS:

1. Genes vs UMI plot (Plot 5):
   - Linear relationship expected for healthy cells
   - Outliers above the line: potential doublets
   - Outliers below: low-complexity cells (stressed/dying)
   - Color gradient shows MT% - healthy cells cluster at low MT%

2. UMI vs MT% plot (Plot 6):
   - Healthy cells: high UMI, low MT%
   - Dying cells: variable UMI, high MT%
   - The paper's 10% MT threshold effectively removes stressed cells

3. Distribution shapes:
   - Normal distributions suggest homogeneous cell populations
   - Bimodal patterns may indicate distinct cell types or quality states
   - Long tails often represent technical artifacts or rare cell states

Paper connection: The strict QC criteria (UMI >2000, genes 500-7000, MT <10%) 
ensure that downstream analysis focuses on high-quality epithelial and fibroblast cells.

In [6]:
# =============================================================================
# Step 5: Apply Quality Control Filters
# =============================================================================

"""
Filtering Strategy (based on the paper):
1. UMI counts > 2000 (removes low-quality cells and empty droplets)
2. Genes detected: 500-7000 (removes empty droplets and potential doublets)
3. MT% < 10% (removes dying/stressed cells)
4. Remove predicted doublets

BIOLOGICAL RATIONALE:
- These thresholds preserve metabolically active, transcriptionally diverse cells
- Particularly important for tissue samples where dissociation can stress cells
- The paper's focus on epithelial/fibroblast requires high-quality cells for accurate clustering
"""

print(f"Before filtering: {adata.n_obs} cells")

# Apply filters step by step so we can track what's being removed
# Filter 1: UMI counts
sc.pp.filter_cells(adata, min_counts=2000)
print(f"After UMI filter (>2000): {adata.n_obs} cells")

# Filter 2: Number of genes (remove cells with too few or too many genes)
sc.pp.filter_cells(adata, min_genes=500)  # Remove cells with <500 genes
adata = adata[adata.obs.n_genes_by_counts < 7000, :].copy()  # Remove cells with >7000 genes
print(f"After gene count filter (500-7000): {adata.n_obs} cells")

# Filter 3: Mitochondrial gene percentage
if adata.obs['pct_counts_mt'].max() > 0:
    # Only apply MT filter if we actually have MT gene data
    adata = adata[adata.obs.pct_counts_mt < 10, :].copy()
    print(f"After MT filter (<10%): {adata.n_obs} cells")
else:
    print(f"Skipping MT filter (no MT genes detected): {adata.n_obs} cells")

# Filter 4: Remove doublets
adata = adata[~adata.obs.predicted_doublet, :].copy()
print(f"After doublet removal: {adata.n_obs} cells")

# Also filter genes (remove genes expressed in very few cells)
sc.pp.filter_genes(adata, min_cells=3)  # Remove genes expressed in <3 cells
print(f"After gene filtering: {adata.n_vars} genes")

print(f"\nFinal filtered dataset: {adata.n_obs} cells × {adata.n_vars} genes")

Before filtering: 51140 cells
filtered out 12365 cells that have less than 2000 counts
After UMI filter (>2000): 38775 cells
filtered out 503 cells that have less than 500 genes expressed
After gene count filter (500-7000): 38136 cells
After MT filter (<10%): 32583 cells
After doublet removal: 32583 cells
filtered out 8539 genes that are detected in less than 3 cells
After gene filtering: 24999 genes

Final filtered dataset: 32583 cells × 24999 genes


BIOLOGICAL IMPACT OF FILTERING:
- Typically removes 10-30% of cells, focusing on high-quality populations
- Gene filtering removes very rare transcripts that don't contribute to cell type identification
- The remaining cells should represent viable, transcriptionally active populations
- Essential for accurate downstream clustering and cell type annotation

In [7]:
# =============================================================================
# Step 6: Normalization and Log Transformation
# =============================================================================

"""
Normalization accounts for differences in sequencing depth between cells.
Log transformation stabilizes variance and makes data more normal-like.

BIOLOGICAL NECESSITY:
- Different cells are sequenced to different depths (technical variation)
- Some cells are naturally more transcriptionally active (biological variation)
- Normalization allows fair comparison between cells
- Log transformation handles the wide dynamic range of gene expression
"""

# Save the raw count data (important for differential expression later)
adata.raw = adata

# Normalize to 10,000 reads per cell (standard in the field)
# This makes cells comparable by adjusting for sequencing depth
sc.pp.normalize_total(adata, target_sum=1e4)

# Log transform: log(x + 1) 
# The +1 prevents log(0) and handles zero counts
sc.pp.log1p(adata)

print("Data normalized and log-transformed!")
print("Raw counts saved in adata.raw for later use")

normalizing counts per cell
    finished (0:00:00)
Data normalized and log-transformed!
Raw counts saved in adata.raw for later use


BIOLOGICAL INTERPRETATION:
- Normalization assumes most genes are not differentially expressed between cells
- 10,000 target sum is arbitrary but standard (easier interpretation)
- Log transformation reduces the influence of highly expressed genes
- Essential for comparing expression between cell types accurately

In [8]:
# =============================================================================
# Step 7: Find Highly Variable Genes
# =============================================================================

"""
Highly variable genes (HVGs) are the most informative for distinguishing cell types.
We focus on these to reduce noise and computational burden.

BIOLOGICAL SIGNIFICANCE:
- HVGs capture cell type-specific expression programs
- Housekeeping genes (low variability) are less informative for clustering
- The paper identified 2536 HVGs for epithelial and 2267 for fibroblast populations
- These genes drive the major axes of variation in the data
"""

# Find highly variable genes using Scanpy's default method
# This identifies genes that vary more than expected by chance
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# Plot the results
sc.pl.highly_variable_genes(adata)

n_hvgs = adata.var['highly_variable'].sum()
print(f"Found {n_hvgs} highly variable genes")

# Keep only highly variable genes for downstream analysis
# (we can always go back to all genes using adata.raw)
adata.var['highly_variable_original'] = adata.var['highly_variable'].copy()

extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
Found 2143 highly variable genes


BIOLOGICAL INTERPRETATION OF HVG PLOT:
- X-axis: Mean expression level
- Y-axis: Dispersion (normalized variance)
- Red dots: Selected highly variable genes
- These genes show higher variance than expected from their mean expression
- Often include cell type markers, stress response genes, and developmental regulators

Paper connection: The similar number of HVGs (2000-2500) suggests balanced 
representation of epithelial and fibroblast gene expression programs.

In [9]:
# =============================================================================
# Step 8: Scale Data and Principal Component Analysis (PCA)
# =============================================================================

"""
Scaling: Centers genes to mean=0 and scales to variance=1
PCA: Reduces dimensionality while preserving most variance

BIOLOGICAL PURPOSE:
- Scaling prevents highly expressed genes from dominating the analysis
- PCA identifies the major axes of transcriptional variation
- First PCs often correspond to cell type differences
- Later PCs may capture technical effects or subtle biological processes
"""

# Scale data to unit variance (important for PCA)
# Clip values to prevent extreme outliers from dominating
sc.pp.scale(adata, max_value=10)

# Perform PCA - 50 components is usually sufficient
# PCA finds the directions of maximum variance in the data
sc.tl.pca(adata, svd_solver='arpack', n_comps=50)

# Plot PCA results
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

print("PCA completed with 50 components")

MemoryError: Unable to allocate 6.07 GiB for an array with shape (32583, 24999) and data type float64

BIOLOGICAL INTERPRETATION OF PCA:
- First few PCs typically capture major cell type differences
- PC1 might separate epithelial from fibroblast populations
- Later PCs often capture subtypes within major cell classes
- Variance ratio plot shows how much variation each PC explains
- Usually need 20-50 PCs to capture most biological variation

Paper methodology: Used 50 PCs as input for Harmony correction, 
ensuring comprehensive capture of biological variation.

In [ ]:
# =============================================================================
# Step 9: Batch Correction (if needed)
# =============================================================================

"""
If your data has multiple batches/samples/patients, use Harmony for correction.
This removes technical variation while preserving biological differences.

BIOLOGICAL IMPORTANCE:
- Patient-to-patient variation can mask true cell type differences
- Technical batches can create artificial clusters
- The paper used patient-wise Harmony correction
- Critical for multi-sample studies like GSE194247
"""

# Check what batch information we have
potential_batch_keys = [col for col in adata.obs.columns 
                       if any(keyword in col.lower() for keyword in 
                             ['batch', 'sample', 'patient', 'donor', 'experiment'])]

print(f"Potential batch keys found: {potential_batch_keys}")

# If you have batch effects, uncomment and modify this:
if len(potential_batch_keys) > 0:
    batch_key = potential_batch_keys[0]  # Use the first one found
    print(f"Applying Harmony batch correction using: {batch_key}")
    
    # Apply Harmony batch correction
    sc.external.pp.harmony_integrate(adata, key=batch_key, basis='X_pca', 
                                   adjusted_basis='X_pca_harmony')
    
    # Use Harmony-corrected PCs for downstream analysis
    pca_use = 'X_pca_harmony'
    print("Harmony correction applied!")
    
    """
    BIOLOGICAL SIGNIFICANCE OF BATCH CORRECTION:
    - Removes patient-specific technical effects
    - Preserves shared biological programs across samples
    - Essential for identifying conserved cell types
    - The paper's approach ensures epithelial and fibroblast populations 
      are defined by biology, not technical variation
    """
else:
    print("No batch correction applied")
    pca_use = 'X_pca'


Potential batch keys found: ['sample_id', 'sample']
Applying Harmony batch correction using: sample_id


2025-06-09 17:43:12,954 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-06-09 17:43:16,141 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-06-09 17:43:16,277 - harmonypy - INFO - Iteration 1 of 10
2025-06-09 17:43:23,244 - harmonypy - INFO - Iteration 2 of 10
2025-06-09 17:43:30,358 - harmonypy - INFO - Iteration 3 of 10
2025-06-09 17:43:37,140 - harmonypy - INFO - Converged after 3 iterations


Harmony correction applied!


In [ ]:
# =============================================================================
# Step 10: Build Neighborhood Graph and UMAP
# =============================================================================

"""
Neighborhood graph: Connects similar cells based on gene expression
UMAP: Creates a 2D visualization preserving local neighborhoods

BIOLOGICAL INTERPRETATION:
- Neighborhood graph captures transcriptional similarity
- Cells with similar gene expression profiles are connected
- UMAP preserves both local structure (cell subtypes) and global structure (major cell types)
- Distance in UMAP roughly corresponds to transcriptional similarity
"""
# Build neighborhood graph
# This connects each cell to its most similar neighbors
if 'X_pca_harmony' in adata.obsm.keys():
    # Use batch-corrected PCA if available
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40, use_rep='X_pca_harmony')
    print("Neighborhood graph built using Harmony-corrected PCA")
else:
    # Use regular PCA
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    print("Neighborhood graph built using regular PCA")

# Compute UMAP embedding for visualization
# UMAP preserves both local and global structure
sc.tl.umap(adata)

print("UMAP embedding computed!")

computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:27)
Neighborhood graph built using Harmony-corrected PCA
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm)
    'umap', UMAP parameters (adata.uns) (0:00:12)
UMAP embedding computed!


BIOLOGICAL MEANING OF NEIGHBORHOOD GRAPH:
- Each cell connected to 10 most similar cells
- Creates communities of transcriptionally similar cells
- Foundation for clustering algorithms
- Captures both major cell types and subtle subtypes

UMAP INTERPRETATION:
- Cells close together have similar expression profiles
- Major clusters represent distinct cell types (epithelial, fibroblast, immune, etc.)
- Sub-clusters within major types represent functional states or subtypes
- Continuous transitions may indicate differentiation trajectories

In [ ]:
# =============================================================================
# Step 11: Clustering
# =============================================================================

"""
Leiden clustering groups similar cells together.
Resolution parameter controls cluster granularity:
- Lower values (0.1-0.5): Fewer, broader clusters (major cell types)
- Higher values (1.0-2.0): More, finer clusters (subtypes)

BIOLOGICAL SIGNIFICANCE:
- Clusters ideally represent distinct cell types or states
- Resolution 2.0 (from paper) captures both major types and functional subtypes
- Important for identifying rare cell populations
- Should reflect known biological categories when possible
"""

# Try different resolutions to find the best clustering
resolutions = [0.04,0.1, 0.5, 1.0, 1.5, 2.0]

for res in resolutions:
    sc.tl.leiden(adata, resolution=res, key_added=f'leiden_res_{res}')
    n_clusters = adata.obs[f'leiden_res_{res}'].nunique()
    print(f"Resolution {res}: {n_clusters} clusters")

# Use resolution 2.0 as in the paper (you can change this)
# adata.obs['leiden'] = adata.obs['leiden_res_2.0']
adata.obs['leiden'] = adata.obs['leiden_res_0.04']
n_final_clusters = adata.obs['leiden'].nunique()
print(f"\nUsing resolution 0.04 with {n_final_clusters} clusters for downstream analysis")
# print(f"\nUsing resolution 2.0 with {n_final_clusters} clusters for downstream analysis")

running Leiden clustering
    finished: found 5 clusters and added
    'leiden_res_0.04', the cluster labels (adata.obs, categorical) (0:00:01)
Resolution 0.04: 5 clusters
running Leiden clustering
    finished: found 7 clusters and added
    'leiden_res_0.1', the cluster labels (adata.obs, categorical) (0:00:01)
Resolution 0.1: 7 clusters
running Leiden clustering
    finished: found 19 clusters and added
    'leiden_res_0.5', the cluster labels (adata.obs, categorical) (0:00:02)
Resolution 0.5: 19 clusters
running Leiden clustering
    finished: found 33 clusters and added
    'leiden_res_1.0', the cluster labels (adata.obs, categorical) (0:00:03)
Resolution 1.0: 33 clusters
running Leiden clustering
    finished: found 44 clusters and added
    'leiden_res_1.5', the cluster labels (adata.obs, categorical) (0:00:03)
Resolution 1.5: 44 clusters
running Leiden clustering
    finished: found 53 clusters and added
    'leiden_res_2.0', the cluster labels (adata.obs, categorical) (0:00:03

BIOLOGICAL INTERPRETATION OF CLUSTERING RESOLUTION:
- Resolution 0.5: Major cell types (epithelial, fibroblast, immune)
- Resolution 1.0: Cell types + major functional states
- Resolution 2.0: Detailed subtypes and activation states (paper choice)
- Higher resolutions: Risk of over-clustering technical noise

Paper rationale: Resolution 2.0 allows identification of epithelial and 
fibroblast subtypes while maintaining biological coherence.

In [ ]:
# =============================================================================
# Step 12: Visualization with Biological Context
# =============================================================================

"""
Now let's visualize our results to understand the cell populations

WHAT TO LOOK FOR BIOLOGICALLY:
- Discrete clusters suggest distinct cell types
- Continuous gradients may indicate differentiation states
- Clusters should correlate with known biological features
- Technical confounders should be distributed across clusters
"""

# Create a comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Single Cell Analysis Results - Biological Interpretation', fontsize=16)

# Plot 1: Clusters
sc.pl.umap(adata, color='leiden', legend_loc='on data', legend_fontsize=8, 
           size=30, ax=axes[0, 0], show=False, frameon=False)
axes[0, 0].set_title(f'Leiden Clustering ({n_final_clusters} clusters)\nBiological Communities')

# Plot 2: Total UMI counts
sc.pl.umap(adata, color='total_counts', size=30, ax=axes[0, 1], 
           show=False, frameon=False)
axes[0, 1].set_title('Total UMI Counts\n(Transcriptional Activity)')

# Plot 3: Number of genes
sc.pl.umap(adata, color='n_genes_by_counts', size=30, ax=axes[0, 2], 
           show=False, frameon=False)
axes[0, 2].set_title('Number of Genes\n(Transcriptional Complexity)')

# Plot 4: MT percentage
if adata.obs['pct_counts_mt'].max() > 0:
    sc.pl.umap(adata, color='pct_counts_mt', size=30, ax=axes[1, 0], 
               show=False, frameon=False)
    axes[1, 0].set_title('Mitochondrial Gene %\n(Metabolic Stress)')
else:
    axes[1, 0].text(0.5, 0.5, 'No MT data\navailable', ha='center', va='center', 
                    transform=axes[1, 0].transAxes, fontsize=12)
    axes[1, 0].set_title('Mitochondrial Gene %\n(Not Available)')

# Plot 5: Batch/sample if available
if len(potential_batch_keys) > 0:
    sc.pl.umap(adata, color=potential_batch_keys[0], size=30, ax=axes[1, 1], 
               show=False, frameon=False)
    axes[1, 1].set_title(f'{potential_batch_keys[0]}\n(Technical Batch)')
else:
    axes[1, 1].text(0.5, 0.5, 'No batch\ninformation', ha='center', va='center', 
                    transform=axes[1, 1].transAxes, fontsize=14)
    axes[1, 1].set_title('Batch Information')

# Plot 6: Doublet scores
sc.pl.umap(adata, color='doublet_score', size=30, ax=axes[1, 2], 
           show=False, frameon=False)
axes[1, 2].set_title('Doublet Scores\n(Technical Artifacts)')

plt.tight_layout()
plt.show()
plt.savefig('./figures/scrnaseq_umap.pdf', bbox_inches='tight')
plt.close()

BIOLOGICAL INTERPRETATION OF UMAP PLOTS:

1. Clustering plot: 
   - Distinct clusters = different cell types/states
   - Bridges between clusters = transitional states
   - Isolated clusters = rare or specialized cell types

2. UMI counts plot:
   - High UMI clusters: metabolically active cells (often epithelial)
   - Lower UMI: quiescent cells or stromal cells
   - Should not strongly correlate with clusters (technical confound if it does)

3. Gene complexity plot:
   - Higher complexity: diverse transcriptional programs
   - Lower complexity: specialized or stressed cells
   - Should reflect cell type biology, not just technical quality

4. MT percentage plot:
   - Should be low and evenly distributed
   - High MT regions suggest filtering artifacts
   - Good QC shows uniform low MT% across clusters

5. Batch plot:
   - Good integration shows mixed batches in each cluster
   - Batch-specific clusters indicate incomplete correction
   - Important for multi-patient studies like the paper

Paper connection: The UMAP should show distinct epithelial and fibroblast 
populations, potentially with immune and other stromal cells mixed in.

In [ ]:
# =============================================================================
# Step 13: Find Marker Genes with Biological Context
# =============================================================================

"""
Marker genes help us identify what cell types each cluster represents.
We'll find genes that are specifically expressed in each cluster.

BIOLOGICAL SIGNIFICANCE:
- Marker genes define cell type identity
- Should match known biology (e.g., KRT genes for epithelial cells)
- Can reveal novel cell states or subtypes
- Essential for connecting clusters to biological knowledge
"""

print("Finding marker genes for each cluster...")

# Find marker genes using Wilcoxon rank-sum test
# This compares each cluster vs all others
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', use_raw=True)

# Look at top marker genes
sc.pl.rank_genes_groups(adata, n_genes=5, sharey=False, show=False)
plt.suptitle('Top Marker Genes per Cluster\n(Key for Biological Annotation)')
plt.show()
plt.savefig('./figures/scrnaseq_expected_marker_genes_per_leiden_cluster.pdf', bbox_inches='tight')
plt.close()

# Get marker genes as a dataframe for easier viewing
marker_genes = sc.get.rank_genes_groups_df(adata, None)
print("Top 3 marker genes per cluster:")
for cluster in sorted(adata.obs['leiden'].unique()):
    cluster_markers = marker_genes[marker_genes['group'] == cluster].head(3)
    genes = ', '.join(cluster_markers['names'].tolist())
    print(f"Cluster {cluster}: {genes}")

Finding marker genes for each cluster...
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:17)
Top 3 marker genes per cluster:
Cluster 0: DCN, LUM, RARRES2
Cluster 1: SPINT2, KRT19, KRT8
Cluster 2: PECAM1, RAMP2, PLVAP
Cluster 3: NDUFA4L2, SPARCL1, MYL9
Cluster 4: CD74, HLA-DRA, VIM


BIOLOGICAL INTERPRETATION OF YOUR SPECIFIC MARKER GENES:

Expected marker patterns for your tissue dataset:

ENDOTHELIAL MARKERS:
- PECAM1 (CD31): pan-endothelial marker
- VWF: von Willebrand factor (blood vessel function)
- CDH5 (VE-cadherin): endothelial junctions
- FLT1, KDR: VEGF receptors (angiogenesis)
- CLDN5: tight junction protein

EPITHELIAL MARKERS:
- KRT genes (KRT8, KRT18, KRT19): epithelial cytoskeleton
- EPCAM: epithelial cell adhesion molecule
- CDH1 (E-cadherin): epithelial adherens junctions
- CLDN genes: tight junction proteins
- MUC genes: mucin proteins (secretory epithelium)

FIBROBLAST MARKERS:
- COL1A1, COL3A1: collagen (ECM production)
- FN1: fibronectin (ECM component)
- ACTA2: smooth muscle actin (activated fibroblasts)
- PDGFRA: platelet-derived growth factor receptor
- VIM: vimentin (mesenchymal marker)

SCHWANN CELL MARKERS:
- MPZ: myelin protein zero (major myelin protein)
- PMP22: peripheral myelin protein 22
- S100B: calcium-binding protein (glial marker)
- SOX10: transcription factor (neural crest)
- NCAM1: neural cell adhesion molecule

STELLATE CELL MARKERS:
- ACTA2: smooth muscle actin (activation marker)
- PDGFRB: platelet-derived growth factor receptor beta
- RGS5: regulator of G-protein signaling (pericyte marker)
- VIM: vimentin
- LRAT: lecithin retinol acyltransferase (hepatic stellate specific)

TISSUE CONTEXT:
This cell type combination is characteristic of:
- Liver tissue (hepatic stellate cells + hepatocytes as epithelial)
- Pancreatic tissue (pancreatic stellate cells + ductal epithelium)
- Complex organs with rich innervation and vasculature

Paper connection: The analysis should reveal distinct populations of these
five major cell types, potentially with functional subtypes within each class.

In [ ]:
# =============================================================================
# Step 14: Load and Use Your Marker Gene Files
# =============================================================================

"""
Now let's use your provided marker gene files to annotate cell types
"""

try:
    # Load your marker gene files
    marker_major = pd.read_csv("marker_dict_major.csv")
    marker_subclusters = pd.read_csv("marker_dict_subclusters.csv")
    
    print("Marker gene files loaded successfully!")
    print(f"Major cell types: {len(marker_major)} entries")
    print(f"Subclusters: {len(marker_subclusters)} entries")
    
    # Display the structure of your marker files
    print("\nMajor markers structure:")
    print(marker_major.head())
    print(f"Columns: {list(marker_major.columns)}")
    
    print("\nSubcluster markers structure:")
    print(marker_subclusters.head())
    print(f"Columns: {list(marker_subclusters.columns)}")
    
    # Show the cell types present in your data
    if 'celltype' in marker_major.columns:
        unique_celltypes = marker_major['celltype'].unique()
        print(f"\nCell types in marker file: {list(unique_celltypes)}")
    
    """
    BIOLOGICAL CONTEXT - Your Dataset Cell Types:
    
    ENDOTHELIAL CELLS:
    - Line blood and lymphatic vessels
    - Key markers: PECAM1 (CD31), VWF, CDH5 (VE-cadherin), FLT1
    - Function: vascular barrier, angiogenesis, blood flow regulation
    
    EPITHELIAL CELLS:
    - Form protective barriers in organs
    - Key markers: KRT genes, EPCAM, CDH1 (E-cadherin), CLDN genes
    - Function: barrier formation, secretion, absorption
    
    FIBROBLAST CELLS:
    - Produce extracellular matrix and provide structural support
    - Key markers: COL1A1, COL3A1, FN1, ACTA2, PDGFRA
    - Function: ECM production, wound healing, tissue remodeling
    
    SCHWANN CELLS:
    - Peripheral nervous system support cells
    - Key markers: MPZ, PMP22, S100B, NCAM1, SOX10
    - Function: myelination of peripheral nerves, nerve regeneration
    
    STELLATE CELLS:
    - Tissue-specific stromal cells (hepatic or pancreatic stellate)
    - Key markers: ACTA2, PDGFRB, VIM, RGS5 (hepatic: LRAT, GFAP)
    - Function: ECM regulation, fibrosis, tissue homeostasis
    
    This combination suggests analysis of a complex tissue with vascular,
    epithelial, stromal, and neural components - possibly liver, pancreas,
    or other complex organ system.
    """
    
except FileNotFoundError:
    print("Marker gene files not found. Please ensure they are in the current directory.")
    print("Expected files: marker_dict_major.csv, marker_dict_subclusters.csv")
    
    # Create a simple annotation based on top marker genes
    marker_major = None
    marker_subclusters = None

Marker gene files loaded successfully!
Major cell types: 50 entries
Subclusters: 51 entries

Major markers structure:
      celltype   markers
0  Endothelial      FLT4
1  Endothelial     MYCT1
2  Endothelial  ARHGEF15
3  Endothelial     CCL14
4  Endothelial    GIMAP6
Columns: ['celltype', 'markers']

Subcluster markers structure:
    cluster  genes
0  Ep_FXYD2   AMBP
1  Ep_FXYD2  DCDC2
2  Ep_FXYD2  TRPV6
3  Ep_FXYD2  FXYD2
4  Ep_FXYD2  SFRP5
Columns: ['cluster', 'genes']

Cell types in marker file: ['Endothelial', 'Epithelial', 'Fibroblast', 'Schwann', 'Stellate']


In [ ]:
# =============================================================================
# Step 15: Cell Type Annotation with Biological Validation
# =============================================================================

"""
Annotate clusters with cell types based on marker gene expression

BIOLOGICAL ANNOTATION STRATEGY:
1. Compare cluster markers with known cell type signatures
2. Validate annotations with multiple markers per cell type
3. Consider expression levels, not just presence/absence
4. Account for tissue-specific marker variation
"""

if marker_major is not None:
    # Advanced annotation strategy based on marker overlap
    # Using the correct column names from your CSV files
    
    cluster_annotations = {}
    annotation_scores = {}
    
    # Get top markers for each cluster
    for cluster in sorted(adata.obs['leiden'].unique()):
        cluster_markers = marker_genes[marker_genes['group'] == cluster].head(200)
        cluster_gene_set = set(cluster_markers['names'].tolist())
        
        best_match = "Unknown"
        best_score = 0
        all_scores = {}
        
        # Compare with known cell type markers using correct column names
        for _, row in marker_major.iterrows():
            if 'celltype' in marker_major.columns and 'markers' in marker_major.columns:
                cell_type = row['celltype']
                markers = str(row['markers']).split(',') if pd.notna(row['markers']) else []
                markers = [m.strip() for m in markers]
                
                # Calculate overlap score
                overlap = len(cluster_gene_set.intersection(set(markers)))
                overlap_score = overlap / len(markers) if len(markers) > 0 else 0
                all_scores[cell_type] = overlap_score
                
                if overlap > best_score:
                    best_score = overlap
                    best_match = cell_type
        
        cluster_annotations[cluster] = best_match if best_score > 0 else f"Cluster_{cluster}"
        annotation_scores[cluster] = all_scores
    
    # Add annotations to data
    adata.obs['cell_type'] = adata.obs['leiden'].map(cluster_annotations)
    
    print("\nCluster annotations with confidence scores:")
    for cluster, cell_type in cluster_annotations.items():
        n_cells = (adata.obs['leiden'] == cluster).sum()
        scores = annotation_scores[cluster]
        
        # Show top 3 scoring cell types for this cluster
        top_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]
        score_str = ", ".join([f"{ct}: {score:.2f}" for ct, score in top_scores])
        
        print(f"Cluster {cluster}: {cell_type} ({n_cells} cells)")
        print(f"  Confidence scores - {score_str}")

else:
    # Simple annotation based on cluster numbers
    adata.obs['cell_type'] = 'Cluster_' + adata.obs['leiden'].astype(str)

adata_integration = adata.copy()
adata_integration.write('./data_and_results/scRNA_seq/GSE194247/adata_all_preprocessed_major_cell_type.h5ad', compression='gzip')


Cluster annotations with confidence scores:
Cluster 0: Fibroblast (13727 cells)
  Confidence scores - Fibroblast: 1.00, Endothelial: 0.00, Epithelial: 0.00
Cluster 1: Epithelial (13387 cells)
  Confidence scores - Endothelial: 0.00, Epithelial: 0.00, Fibroblast: 0.00
Cluster 2: Endothelial (2755 cells)
  Confidence scores - Endothelial: 0.00, Epithelial: 0.00, Fibroblast: 0.00
Cluster 3: Stellate (2428 cells)
  Confidence scores - Stellate: 1.00, Endothelial: 0.00, Epithelial: 0.00
Cluster 4: Schwann (286 cells)
  Confidence scores - Schwann: 1.00, Endothelial: 0.00, Epithelial: 0.00


BIOLOGICAL VALIDATION OF YOUR SPECIFIC CELL TYPES:

ENDOTHELIAL CELLS should show:
- High PECAM1 (CD31), VWF, CDH5 expression
- Often form distinct, compact clusters
- May show vessel-type heterogeneity (arterial vs venous vs capillary)

EPITHELIAL CELLS should show:
- Strong KRT gene expression (KRT8, KRT18, KRT19)
- EPCAM, CDH1 positivity
- May separate by epithelial subtype (basal vs luminal)

FIBROBLAST CELLS should show:
- High collagen expression (COL1A1, COL3A1)
- FN1, PDGFRA expression
- May include activated/quiescent states

SCHWANN CELLS should show:
- MPZ, PMP22 (myelin proteins)
- S100B, SOX10 (neural crest markers)
- Likely smaller population unless nervous tissue-rich sample

STELLATE CELLS should show:
- ACTA2 (smooth muscle actin)
- PDGFRB, VIM expression
- Tissue-specific markers (LRAT for hepatic, etc.)

The presence of these five cell types suggests a complex tissue environment,
possibly from liver, pancreas, or other organs with rich vascular, epithelial,
stromal, and neural innervation.

In [ ]:
# =============================================================================
# Step 16: Final Visualization with Biological Interpretation
# =============================================================================

"""
Create final publication-ready plots with biological context
"""

# Plot with cell type annotations
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Clusters
sc.pl.umap(adata, color='leiden', legend_loc='on data', legend_fontsize=10, 
           size=40, ax=axes[0], show=False, frameon=False)
axes[0].set_title('Leiden Clusters\n(Computational Communities)', fontsize=14, fontweight='bold')

# Cell types
sc.pl.umap(adata, color='cell_type', legend_loc='right margin', legend_fontsize=10, 
           size=40, ax=axes[1], show=False, frameon=False)
axes[1].set_title('Cell Type Annotation\n(Biological Identity)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()
plt.savefig('./figures/scrnaseq_cell_type_annotations.pdf', bbox_inches='tight')
plt.close()


BIOLOGICAL INTERPRETATION OF FINAL PLOTS FOR YOUR CELL TYPES:

EXPECTED SPATIAL PATTERNS in UMAP:
1. Endothelial clusters: Often compact, may show vessel-type heterogeneity
2. Epithelial clusters: Should be well-separated, may show polarity-related subtypes
3. Fibroblast clusters: May show activation state heterogeneity (quiescent vs activated)
4. Schwann clusters: Typically smaller, distinct from other cell types
5. Stellate clusters: May overlap partially with fibroblasts (shared mesenchymal origin)

QUALITY INDICATORS FOR YOUR SYSTEM:
- Clear boundaries between major cell types
- Endothelial cells should cluster separately from epithelial
- Fibroblast and stellate cells may show some overlap (both mesenchymal)
- Schwann cells should be distinct (neural crest origin)
- Within-type heterogeneity reflecting functional states

EXPECTED CELL TYPE RELATIONSHIPS:
- Endothelial-epithelial: may show some continuity (vascular-parenchymal interface)
- Fibroblast-stellate: likely show proximity (both ECM-producing)
- Schwann cells: isolated clusters (specialized neural function)

BIOLOGICAL VALIDATION CRITERIA:
- Each cluster should express appropriate lineage markers
- Activation markers (ACTA2) should identify activated stromal cells
- Tissue-specific markers should match expected organ type
- Cell proportions should reflect normal tissue architecture

PATHOLOGICAL INDICATORS (if disease samples):
- Expanded stellate cell populations (fibrosis)
- Activated fibroblast signatures (tissue remodeling)
- Endothelial dysfunction markers
- Epithelial dedifferentiation signatures

Your final plots should show five distinct cell type territories with
appropriate marker expression and biological relationships that reflect
the complex tissue architecture of your organ system.

In [ ]:
# =============================================================================
# Step 17: Expression Heatmap with Biological Context
# =============================================================================

"""
Visualize marker gene expression across clusters

BIOLOGICAL INSIGHTS FROM HEATMAPS:
- Clear on/off patterns indicate well-defined cell types
- Gradual transitions suggest differentiation states
- Co-expression patterns reveal functional modules
- Cross-cluster expression may indicate shared functions
"""

# Get top 3 marker genes per cluster for heatmap
top_markers = []
marker_annotations = {}

for cluster in sorted(adata.obs['leiden'].unique()):
    cluster_markers = marker_genes[marker_genes['group'] == cluster].head(150)
    for gene in cluster_markers['names']:
        if gene in adata.var_names and gene not in top_markers:
            top_markers.append(gene)
            # Annotate gene function based on your five cell types
            if gene in ['PECAM1', 'VWF', 'CDH5', 'FLT1', 'KDR', 'CLDN5']:
                marker_annotations[gene] = 'Endothelial'
            elif gene.startswith(('KRT', 'CK')) or gene in ['EPCAM', 'CDH1']:
                marker_annotations[gene] = 'Epithelial'
            elif gene.startswith(('COL', 'FN')) or gene in ['ACTA2', 'PDGFRA', 'VIM']:
                marker_annotations[gene] = 'Fibroblast'
            elif gene in ['MPZ', 'PMP22', 'S100B', 'SOX10', 'NCAM1']:
                marker_annotations[gene] = 'Schwann'
            elif gene in ['PDGFRB', 'RGS5', 'LRAT', 'GFAP'] or (gene == 'ACTA2'):
                marker_annotations[gene] = 'Stellate'
            elif gene.startswith('MT-'):
                marker_annotations[gene] = 'Mitochondrial'
            else:
                marker_annotations[gene] = 'Other'

# Limit to 20 genes for readability
#display_genes = top_markers[:20]
display_genes = ['CDH5', 'EPCAM', 'PDGFRA', 'SOX10', 'RGS5']

if len(display_genes) > 0:
    # Create heatmap
    # sc.pl.heatmap(adata, display_genes, groupby='leiden', use_raw=True, 
    #               cmap='viridis', figsize=(12, 8), show=False)
    # sc.pl.violin(adata, display_genes, groupby='leiden', use_raw=True, 
    #               stripplot=False, jitter=False, rotation=0, show=False)
    sc.pl.violin(adata, display_genes, groupby='cell_type', use_raw=True, 
                stripplot=False, jitter=False, rotation=90, show=False)
    plt.title('Top Marker Genes Expression by Cluster\n(Biological Signatures)', 
              fontsize=14, fontweight='bold')
    plt.show()
    plt.savefig('./figures/scrnaseq_expression_violin.pdf', bbox_inches='tight')
    plt.close()
    
    # Print gene annotations
    print("\nMarker gene biological annotations:")
    for gene in display_genes:
        annotation = marker_annotations.get(gene, 'Unknown')
        print(f"{gene}: {annotation}")


Marker gene biological annotations:
CDH5: Endothelial
EPCAM: Epithelial
PDGFRA: Fibroblast
SOX10: Schwann
RGS5: Stellate


BIOLOGICAL INTERPRETATION OF EXPRESSION HEATMAP FOR YOUR CELL TYPES:

WHAT TO LOOK FOR:
1. Block patterns indicating distinct cell type signatures
2. Gradient patterns suggesting activation states or differentiation
3. Shared markers revealing functional relationships
4. Cell type-specific exclusive markers

ENDOTHELIAL SIGNATURES:
- PECAM1, VWF, CDH5: vascular identity
- FLT1, KDR: angiogenic potential
- CLDN5: barrier function

EPITHELIAL SIGNATURES:
- KRT genes: structural identity and subtype
- EPCAM: epithelial lineage commitment
- CDH1: epithelial junctions and polarity

FIBROBLAST SIGNATURES:
- Collagen genes: ECM production capacity
- ACTA2: activation/myofibroblast state
- PDGFRA: growth factor responsiveness

SCHWANN CELL SIGNATURES:
- MPZ, PMP22: myelination capacity
- S100B: glial cell identity
- SOX10: neural crest origin

STELLATE CELL SIGNATURES:
- ACTA2: activation state
- PDGFRB: growth factor signaling
- LRAT: hepatic stellate specificity (if present)

QUALITY INDICATORS FOR YOUR SYSTEM:
- High specificity: each cell type should have unique markers
- Biological coherence: co-expressed genes should be functionally related
- Activation states: ACTA2 should mark activated fibroblasts/stellate cells
- Tissue context: marker patterns should match expected organ biology

EXPECTED RELATIONSHIPS:
- Fibroblast and stellate cells may share some ECM markers
- Endothelial cells should be highly distinct
- Epithelial subtypes may show graded KRT expression
- Schwann cells should have unique neural markers

This heatmap should reveal the molecular basis of your five-cell-type
system and validate the biological accuracy of your clustering approach.

In [ ]:
# =============================================================================
# Step 18: Advanced Biological Analysis
# =============================================================================

"""
Additional biological insights from the data
"""

# Analyze cell cycle if genes are available
cell_cycle_genes = [g for g in adata.var_names if any(cc in g for cc in ['CCND', 'CCNE', 'MKI67', 'TOP2A'])]
if len(cell_cycle_genes) > 0:
    print(f"\nFound {len(cell_cycle_genes)} cell cycle genes")
    
    # Plot cell cycle markers
    if len(cell_cycle_genes) >= 4:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        for i, gene in enumerate(cell_cycle_genes[:4]):
            row, col = i // 2, i % 2
            sc.pl.umap(adata, color=gene, size=20, ax=axes[row, col], 
                      show=False, frameon=False)
            axes[row, col].set_title(f'{gene} (Cell Cycle)')
        plt.suptitle('Cell Cycle Gene Expression\n(Proliferative Activity)')
        plt.tight_layout()
        plt.show()

# Analyze tissue-specific markers if available
epithelial_markers = [g for g in adata.var_names if g.startswith('KRT') or g in ['EPCAM', 'CDH1']]
fibroblast_markers = [g for g in adata.var_names if g.startswith('COL') or g in ['FN1', 'ACTA2']]

print(f"\nTissue-specific markers found:")
print(f"Epithelial markers: {len(epithelial_markers)}")
print(f"Fibroblast markers: {len(fibroblast_markers)}")

# Analyze tissue-specific markers for your five major cell types
endothelial_markers = [g for g in adata.var_names if g in ['PECAM1', 'VWF', 'CDH5', 'FLT1', 'KDR', 'CLDN5']]
epithelial_markers = [g for g in adata.var_names if g.startswith('KRT') or g in ['EPCAM', 'CDH1']]
fibroblast_markers = [g for g in adata.var_names if g.startswith('COL') or g in ['FN1', 'ACTA2', 'PDGFRA', 'VIM']]
schwann_markers = [g for g in adata.var_names if g in ['MPZ', 'PMP22', 'S100B', 'SOX10', 'NCAM1']]
stellate_markers = [g for g in adata.var_names if g in ['PDGFRB', 'RGS5', 'LRAT', 'GFAP'] or (g == 'ACTA2')]

print(f"\nTissue-specific markers found:")
print(f"Endothelial markers: {len(endothelial_markers)} - {endothelial_markers}")
print(f"Epithelial markers: {len(epithelial_markers)} - {epithelial_markers[:10]}")  # Show first 10
print(f"Fibroblast markers: {len(fibroblast_markers)} - {fibroblast_markers}")
print(f"Schwann cell markers: {len(schwann_markers)} - {schwann_markers}")
print(f"Stellate cell markers: {len(stellate_markers)} - {stellate_markers}")

# Create signature scores for each major cell type
if len(endothelial_markers) >= 2:
    sc.tl.score_genes(adata, endothelial_markers, score_name='endothelial_score')
if len(epithelial_markers) >= 3:
    sc.tl.score_genes(adata, epithelial_markers[:5], score_name='epithelial_score')  # Use top 5
if len(fibroblast_markers) >= 3:
    sc.tl.score_genes(adata, fibroblast_markers[:5], score_name='fibroblast_score')
if len(schwann_markers) >= 2:
    sc.tl.score_genes(adata, schwann_markers, score_name='schwann_score')
if len(stellate_markers) >= 2:
    sc.tl.score_genes(adata, stellate_markers, score_name='stellate_score')

# Plot signature scores
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Cell Type Signature Scores\n(Biological Identity Validation)', fontsize=16)

# Plot each signature if available
signatures = ['endothelial_score', 'epithelial_score', 'fibroblast_score', 
              'schwann_score', 'stellate_score']
titles = ['Endothelial Signature\n(PECAM1, VWF, CDH5)', 
          'Epithelial Signature\n(KRT genes, EPCAM)', 
          'Fibroblast Signature\n(Collagen, ECM genes)',
          'Schwann Cell Signature\n(MPZ, PMP22, S100B)',
          'Stellate Cell Signature\n(PDGFRB, RGS5, ACTA2)']

for i, (sig, title) in enumerate(zip(signatures, titles)):
    row, col = i // 3, i % 3
    if sig in adata.obs.columns:
        sc.pl.umap(adata, color=sig, size=30, ax=axes[row, col], 
                  show=False, frameon=False, cmap='viridis')
        axes[row, col].set_title(title)
    else:
        axes[row, col].text(0.5, 0.5, f'Insufficient\n{sig.replace("_score", "")} markers', 
                           ha='center', va='center', transform=axes[row, col].transAxes, fontsize=12)
        axes[row, col].set_title(title)

# Last subplot for cell type annotations
sc.pl.umap(adata, color='cell_type', legend_loc='right margin', legend_fontsize=8, 
           size=30, ax=axes[1, 2], show=False, frameon=False)
axes[1, 2].set_title('Final Cell Type\nAnnotations')

plt.tight_layout()
plt.show()
plt.savefig('./figures/scrnaseq_cell_type_signature_score.pdf', bbox_inches='tight')
plt.close()



Found 8 cell cycle genes

Tissue-specific markers found:
Epithelial markers: 67
Fibroblast markers: 57

Tissue-specific markers found:
Endothelial markers: 6 - ['KDR', 'VWF', 'FLT1', 'CDH5', 'PECAM1', 'CLDN5']
Epithelial markers: 67 - ['KRTCAP2', 'KRTCAP3', 'EPCAM', 'KRTAP5-AS1', 'KRTAP5-1', 'KRTAP5-2', 'KRTAP5-5', 'KRTAP5-7', 'KRTAP5-9', 'KRTAP5-10']
Fibroblast markers: 59 - ['COL16A1', 'COL8A2', 'COL9A2', 'COL24A1', 'COL11A1', 'COLGALT2', 'COLEC11', 'COL3A1', 'COL5A2', 'FN1', 'COL4A4', 'COL4A3', 'COL6A3', 'COLQ', 'COL7A1', 'COL8A1', 'COL6A5', 'COL6A6', 'PDGFRA', 'COL25A1', 'COL4A3BP', 'COL23A1', 'COL11A2', 'COL21A1', 'COL19A1', 'COL9A1', 'COL12A1', 'COL10A1', 'COL28A1', 'COL1A2', 'COL26A1', 'COL4A6', 'COL4A5', 'COLEC10', 'COL14A1', 'COL22A1', 'COL15A1', 'COL27A1', 'COL5A1', 'COLCA1', 'COLCA2', 'VIM', 'COL13A1', 'ACTA2', 'COL17A1', 'COL2A1', 'COL4A1', 'COL4A2', 'COL4A2-AS1', 'COL1A1', 'COLEC12', 'COL9A3', 'COL20A1', 'COL5A3', 'COLGALT1', 'COL18A1', 'COL18A1-AS1', 'COL6A1', 'COL6A2']


BIOLOGICAL INSIGHTS FROM YOUR SIGNATURE ANALYSIS:

ENDOTHELIAL SIGNATURE:
- High scores should identify vascular clusters
- May reveal arterial vs venous vs capillary subtypes
- PECAM1 is the most reliable pan-endothelial marker

EPITHELIAL SIGNATURE:
- Should identify epithelial compartments
- KRT expression patterns may reveal epithelial subtypes
- EPCAM distinguishes epithelial from mesenchymal cells

FIBROBLAST SIGNATURE:
- High collagen expression indicates active ECM production
- ACTA2+ cells are activated/myofibroblasts
- May show heterogeneity based on activation state

SCHWANN CELL SIGNATURE:
- MPZ and PMP22 are specific myelin proteins
- SOX10 is a key neural crest transcription factor
- Usually smaller population unless nerve-rich tissue

STELLATE CELL SIGNATURE:
- PDGFRB+ indicates activated stellate cells
- RGS5 overlaps with pericyte markers
- LRAT specifically marks hepatic stellate cells if present

TISSUE INTERPRETATION:
Your combination of cell types suggests:
- Complex organ with multiple tissue compartments
- Rich vascular network (endothelial prominence)
- Epithelial structures (ducts, acini, or parenchyma)
- Stromal support (fibroblasts and stellate cells)
- Neural innervation (Schwann cells)

This pattern is most consistent with liver, pancreas, or other
complex parenchymal organs analyzed in the original paper.

In [ ]:
# =============================================================================
# Step 19: Summary Statistics with Biological Context
# =============================================================================

"""
Generate summary statistics and save your results
"""

# Print final summary
print("\n" + "="*80)
print("ANALYSIS COMPLETE - BIOLOGICAL SUMMARY")
print("="*80)
print(f"Final dataset: {adata.n_obs:,} cells × {adata.n_vars:,} genes")
print(f"Number of clusters: {adata.obs['leiden'].nunique()}")
print(f"Highly variable genes: {adata.var['highly_variable'].sum():,}")

# Cell counts per cluster with biological context
print("\nCells per cluster (with biological annotations):")
cluster_counts = adata.obs['leiden'].value_counts().sort_index()
for cluster, count in cluster_counts.items():
    cell_type = adata.obs[adata.obs['leiden'] == cluster]['cell_type'].iloc[0]
    percentage = (count / adata.n_obs) * 100
    print(f"  Cluster {cluster} ({cell_type}): {count:,} cells ({percentage:.1f}%)")

# Biological composition analysis for your five major cell types
if 'cell_type' in adata.obs.columns:
    print("\nCell type composition (Expected: Endothelial, Epithelial, Fibroblast, Schwann, Stellate):")
    cell_type_counts = adata.obs['cell_type'].value_counts()
    total_cells = adata.n_obs
    
    for cell_type, count in cell_type_counts.items():
        percentage = (count / total_cells) * 100
        print(f"  {cell_type}: {count:,} cells ({percentage:.1f}%)")
    
    # Biological interpretation of proportions
    print("\nBiological interpretation of cell type proportions:")
    
    for cell_type, count in cell_type_counts.items():
        percentage = (count / total_cells) * 100
        
        if 'endothelial' in cell_type.lower():
            if percentage > 15:
                print(f"  • High endothelial content ({percentage:.1f}%) - suggests highly vascularized tissue")
            elif percentage > 5:
                print(f"  • Normal endothelial content ({percentage:.1f}%) - typical vascular density")
            else:
                print(f"  • Low endothelial content ({percentage:.1f}%) - may be less vascularized region")
                
        elif 'epithelial' in cell_type.lower():
            if percentage > 30:
                print(f"  • High epithelial content ({percentage:.1f}%) - epithelial-rich tissue (liver, pancreas)")
            elif percentage > 10:
                print(f"  • Moderate epithelial content ({percentage:.1f}%) - balanced tissue composition")
            else:
                print(f"  • Low epithelial content ({percentage:.1f}%) - stromal-rich sample")
                
        elif 'fibroblast' in cell_type.lower():
            if percentage > 25:
                print(f"  • High fibroblast content ({percentage:.1f}%) - fibrotic or stromal-rich tissue")
            elif percentage > 10:
                print(f"  • Normal fibroblast content ({percentage:.1f}%) - healthy stromal compartment")
            else:
                print(f"  • Low fibroblast content ({percentage:.1f}%) - epithelial-dominant tissue")
                
        elif 'schwann' in cell_type.lower():
            if percentage > 5:
                print(f"  • High Schwann cell content ({percentage:.1f}%) - richly innervated tissue")
            else:
                print(f"  • Normal Schwann cell content ({percentage:.1f}%) - typical neural innervation")
                
        elif 'stellate' in cell_type.lower():
            if percentage > 10:
                print(f"  • High stellate cell content ({percentage:.1f}%) - activated stellate population")
            else:
                print(f"  • Normal stellate cell content ({percentage:.1f}%) - quiescent stellate cells")

    # Suggest tissue type based on composition
    epithelial_pct = sum([count for ct, count in cell_type_counts.items() 
                         if 'epithelial' in ct.lower()]) / total_cells * 100
    endothelial_pct = sum([count for ct, count in cell_type_counts.items() 
                          if 'endothelial' in ct.lower()]) / total_cells * 100
    stellate_pct = sum([count for ct, count in cell_type_counts.items() 
                       if 'stellate' in ct.lower()]) / total_cells * 100
    
    print(f"\nTissue type prediction based on cell composition:")
    if epithelial_pct > 25 and stellate_pct > 5:
        print("  → Likely LIVER tissue (high epithelial + stellate cells)")
    elif epithelial_pct > 20 and endothelial_pct > 10:
        print("  → Likely PANCREATIC tissue (epithelial + vascular rich)")
    elif endothelial_pct > 15:
        print("  → Highly VASCULARIZED tissue (rich capillary network)")
    else:
        print("  → COMPLEX ORGAN tissue (mixed parenchymal/stromal)")


# Save processed data
adata.write('./data_and_results/scRNA_seq/GSE194247/adata_all_preprocessed_annotated.h5ad')
print(f"\nProcessed data saved to: adata_all_preprocessed_annotated.h5ad")

# Save marker genes to CSV with biological annotations
marker_genes_annotated = marker_genes.copy()
marker_genes_annotated['biological_category'] = marker_genes_annotated['names'].apply(
    lambda x: marker_annotations.get(x, 'Other')
)
marker_genes_annotated.to_csv('./data_and_results/scRNA_seq/GSE194247/marker_genes_by_cluster_annotated.csv', index=False)
print("Annotated marker genes saved to: marker_genes_by_cluster_annotated.csv")

# Save cluster summary with biological metrics
summary_data = []
for cluster in sorted(adata.obs['leiden'].unique()):
    cluster_data = adata.obs[adata.obs['leiden'] == cluster]
    
    # Get top 5 markers for this cluster
    cluster_markers = marker_genes[marker_genes['group'] == cluster].head(5)
    top_genes = ', '.join(cluster_markers['names'].tolist())
    
    # Calculate biological metrics
    avg_mt = cluster_data['pct_counts_mt'].mean()
    avg_complexity = cluster_data['n_genes_by_counts'].mean()
    
    summary_data.append({
        'cluster': cluster,
        'cell_type': cluster_data['cell_type'].iloc[0],
        'n_cells': len(cluster_data),
        'percentage': (len(cluster_data) / adata.n_obs) * 100,
        'mean_genes': avg_complexity,
        'mean_umi': cluster_data['total_counts'].mean(),
        'mean_mt_pct': avg_mt,
        'top_markers': top_genes,
        'quality_score': 'High' if avg_mt < 5 and avg_complexity > 1000 else 'Medium' if avg_mt < 8 else 'Low'
    })

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('./data_and_results/scRNA_seq/GSE194247/cluster_summary_biological.csv', index=False)
print("Biological cluster summary saved to: cluster_summary_biological.csv")

print("\n" + "="*80)
print("FILES CREATED:")
print("- processed_data_complete.h5ad (complete processed dataset)")
print("- marker_genes_by_cluster_annotated.csv (markers with biological categories)")
print("- cluster_summary_biological.csv (summary with biological metrics)")
print("="*80)


ANALYSIS COMPLETE - BIOLOGICAL SUMMARY
Final dataset: 32,583 cells × 24,999 genes
Number of clusters: 5
Highly variable genes: 2,143

Cells per cluster (with biological annotations):
  Cluster 0 (Fibroblast): 13,727 cells (42.1%)
  Cluster 1 (Epithelial): 13,387 cells (41.1%)
  Cluster 2 (Endothelial): 2,755 cells (8.5%)
  Cluster 3 (Stellate): 2,428 cells (7.5%)
  Cluster 4 (Schwann): 286 cells (0.9%)

Cell type composition (Expected: Endothelial, Epithelial, Fibroblast, Schwann, Stellate):
  Fibroblast: 13,727 cells (42.1%)
  Epithelial: 13,387 cells (41.1%)
  Endothelial: 2,755 cells (8.5%)
  Stellate: 2,428 cells (7.5%)
  Schwann: 286 cells (0.9%)

Biological interpretation of cell type proportions:
  • High fibroblast content (42.1%) - fibrotic or stromal-rich tissue
  • High epithelial content (41.1%) - epithelial-rich tissue (liver, pancreas)
  • Normal endothelial content (8.5%) - typical vascular density
  • Normal stellate cell content (7.5%) - quiescent stellate cells
  • N

BIOLOGICAL CONCLUSIONS FOR YOUR FIVE-CELL-TYPE SYSTEM:

EXPECTED RESULTS based on your cell type markers:
1. Clear separation of endothelial, epithelial, fibroblast, Schwann, and stellate populations
2. Multiple subtypes within each major cell class reflecting functional states
3. High-quality cells with appropriate tissue-specific marker expression
4. Cell type proportions consistent with complex organ architecture

VALIDATION STEPS FOR YOUR SPECIFIC SYSTEM:
1. Endothelial cells should express PECAM1, VWF, CDH5
2. Epithelial cells should show strong KRT and EPCAM expression
3. Fibroblasts should express collagen genes and ECM components
4. Schwann cells should express myelin proteins (MPZ, PMP22)
5. Stellate cells should show ACTA2, PDGFRB expression

TISSUE CONTEXT INTERPRETATION:
Your five cell types suggest analysis of:
- Liver tissue: hepatocytes (epithelial), hepatic stellate cells, Kupffer cells
- Pancreatic tissue: ductal/acinar epithelium, pancreatic stellate cells
- Complex parenchymal organ with rich innervation and vasculature

FUNCTIONAL RELATIONSHIPS:
- Endothelial-epithelial: vascular-parenchymal interface
- Fibroblast-stellate: ECM regulation and tissue remodeling
- Schwann cells: neural regulation of organ function
- All types: coordinated tissue homeostasis and response to injury

CLINICAL RELEVANCE:
- Stellate cell activation: fibrosis and disease progression
- Endothelial dysfunction: vascular complications
- Epithelial integrity: organ-specific function
- Neural innervation: metabolic and functional regulation

NEXT STEPS for biological interpretation:
1. Functional enrichment analysis for each cell type
2. Cell-cell communication analysis between types
3. Activation state analysis (especially stellate and fibroblast)
4. Disease signature scoring if pathological samples
5. Comparison with healthy vs diseased tissue datasets

Paper-specific insights: Your analysis should reveal how these five
major cell types interact in tissue homeostasis and potentially
in disease states, providing insights into organ-level biology.

In [ ]:
# =============================================================================
# Step 20: Process CD45-Positive Dataset (GSE235449)
# =============================================================================

"""
CD45+ Processing Strategy:
1. Load raw CD45+ data from GSE235449
2. Apply similar QC and processing as CD45- data
3. Identify immune cell types through clustering and annotation
4. Prepare for integration with CD45- reference

BIOLOGICAL RATIONALE FOR CD45+ CELLS:
- CD45 (PTPRC) is pan-immune marker expressed on all immune cells
- Captures T cells, B cells, macrophages, dendritic cells, neutrophils
- Essential for complete tissue cellular ecosystem representation
- Immune-stromal interactions crucial for tissue function and disease
"""

def process_cd45_positive_data(cd45_pos_path):
    """
    Process CD45-positive immune cell dataset (GSE235449)
    
    Parameters:
    -----------
    cd45_pos_path : str
        Path to raw CD45+ data
    
    Returns:
    --------
    adata_cd45_pos : AnnData
        Processed CD45+ dataset with cell type annotations
    """
    print("Processing CD45-positive immune cell dataset (GSE235449)...")
    
    # Load raw CD45+ data - adjust based on your data format
    # For h5ad files:
    adata_cd45_pos = sc.read_h5ad(cd45_pos_path)
    
    # Alternative loading methods:
    # adata_cd45_pos = sc.read_10x_mtx('path/to/GSE235449/matrix/')  # For 10X mtx files
    # adata_cd45_pos = sc.read_csv('GSE235449_counts.csv')  # For CSV files
    
    print(f"Loaded CD45+ data: {adata_cd45_pos.n_obs} cells × {adata_cd45_pos.n_vars} genes")
    
    # Make gene names unique
    adata_cd45_pos.var_names_make_unique()
    
    # Calculate QC metrics for immune cells
    print("Calculating QC metrics for immune cells...")
    
    # Identify mitochondrial genes
    adata_cd45_pos.var['mt'] = (
        adata_cd45_pos.var_names.str.startswith('MT-') |
        adata_cd45_pos.var_names.str.startswith('mt-') |
        adata_cd45_pos.var_names.str.startswith('MITO')
    )
    
    # Identify ribosomal genes
    adata_cd45_pos.var['ribo'] = adata_cd45_pos.var_names.str.startswith(('RPS', 'RPL'))
    
    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(adata_cd45_pos, percent_top=None, log1p=False, inplace=True)
    
    # Handle MT% if no MT genes detected
    if 'pct_counts_mt' not in adata_cd45_pos.obs.columns:
        n_mt_genes = adata_cd45_pos.var['mt'].sum()
        if n_mt_genes > 0:
            mt_gene_names = adata_cd45_pos.var_names[adata_cd45_pos.var['mt']]
            adata_cd45_pos.obs['pct_counts_mt'] = (
                adata_cd45_pos[:, mt_gene_names].X.sum(axis=1).A1 / 
                adata_cd45_pos.obs['total_counts'] * 100
            )
        else:
            adata_cd45_pos.obs['pct_counts_mt'] = 0.0
    
    print(f"CD45+ QC - Mean genes: {adata_cd45_pos.obs['n_genes_by_counts'].mean():.0f}, "
          f"Mean UMI: {adata_cd45_pos.obs['total_counts'].mean():.0f}, "
          f"Mean MT%: {adata_cd45_pos.obs['pct_counts_mt'].mean():.1f}%")
    
    return adata_cd45_pos

def qc_and_filter_cd45_positive(adata_cd45_pos):
    """
    Apply QC filtering to CD45+ data
    
    Parameters:
    -----------
    adata_cd45_pos : AnnData
        CD45+ data with QC metrics
    
    Returns:
    --------
    adata_filtered : AnnData
        Filtered CD45+ data
    """
    print("Applying QC filters to CD45+ data...")
    print(f"Before filtering: {adata_cd45_pos.n_obs} cells")
    
    # Visualize QC metrics for immune cells
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('CD45+ Immune Cells - Quality Control Metrics', fontsize=14)
    
    # UMI counts
    axes[0, 0].hist(adata_cd45_pos.obs['total_counts'], bins=50, alpha=0.7, color='lightcoral')
    axes[0, 0].axvline(1000, color='red', linestyle='--', label='Min threshold')
    axes[0, 0].set_xlabel('Total UMI counts')
    axes[0, 0].set_ylabel('Number of cells')
    axes[0, 0].set_title('UMI Distribution (Immune Cells)')
    axes[0, 0].legend()
    
    # Gene counts
    axes[0, 1].hist(adata_cd45_pos.obs['n_genes_by_counts'], bins=50, alpha=0.7, color='lightgreen')
    axes[0, 1].axvline(300, color='red', linestyle='--', label='Min threshold')
    axes[0, 1].axvline(5000, color='red', linestyle='--', label='Max threshold')
    axes[0, 1].set_xlabel('Number of genes')
    axes[0, 1].set_ylabel('Number of cells')
    axes[0, 1].set_title('Gene Count Distribution')
    axes[0, 1].legend()
    
    # MT percentage
    if adata_cd45_pos.obs['pct_counts_mt'].max() > 0:
        axes[1, 0].hist(adata_cd45_pos.obs['pct_counts_mt'], bins=50, alpha=0.7, color='gold')
        axes[1, 0].axvline(15, color='red', linestyle='--', label='Max threshold')
        axes[1, 0].set_xlabel('Mitochondrial gene %')
        axes[1, 0].set_ylabel('Number of cells')
        axes[1, 0].set_title('MT Gene % (Immune Cells)')
        axes[1, 0].legend()
    else:
        axes[1, 0].text(0.5, 0.5, 'No MT data\navailable', ha='center', va='center',
                       transform=axes[1, 0].transAxes, fontsize=12)
        axes[1, 0].set_title('MT Gene %')
    
    # UMI vs genes scatter
    axes[1, 1].scatter(adata_cd45_pos.obs['total_counts'], 
                      adata_cd45_pos.obs['n_genes_by_counts'],
                      alpha=0.6, s=1, color='purple')
    axes[1, 1].set_xlabel('Total UMI counts')
    axes[1, 1].set_ylabel('Number of genes')
    axes[1, 1].set_title('UMI vs Gene Complexity')
    
    plt.tight_layout()
    plt.show()
    plt.savefig('./data_and_results/scRNA_seq/GSE235449/figures/scrnaseq_qc_metrics_before_filtering.pdf', bbox_inches='tight')
    plt.close()

    # Apply filters - slightly different thresholds for immune cells
    # Immune cells typically have lower UMI/gene counts than stromal cells
    
    # Filter 1: UMI counts (lower threshold for immune cells)
    sc.pp.filter_cells(adata_cd45_pos, min_counts=1000)
    print(f"After UMI filter (>1000): {adata_cd45_pos.n_obs} cells")
    
    # Filter 2: Gene counts
    sc.pp.filter_cells(adata_cd45_pos, min_genes=300)
    adata_cd45_pos = adata_cd45_pos[adata_cd45_pos.obs.n_genes_by_counts < 5000, :].copy()
    print(f"After gene filter (300-5000): {adata_cd45_pos.n_obs} cells")
    
    # Filter 3: MT percentage (higher threshold for immune cells)
    if adata_cd45_pos.obs['pct_counts_mt'].max() > 0:
        adata_cd45_pos = adata_cd45_pos[adata_cd45_pos.obs.pct_counts_mt < 15, :].copy()
        print(f"After MT filter (<15%): {adata_cd45_pos.n_obs} cells")
    
    # Filter genes
    sc.pp.filter_genes(adata_cd45_pos, min_cells=3)
    print(f"After gene filtering: {adata_cd45_pos.n_vars} genes")
    
    print(f"Final filtered CD45+ dataset: {adata_cd45_pos.n_obs} cells × {adata_cd45_pos.n_vars} genes")
    
    return adata_cd45_pos

def process_and_cluster_cd45_positive(adata_cd45_pos):
    """
    Normalize, process, and cluster CD45+ data to identify immune cell types
    
    Parameters:
    -----------
    adata_cd45_pos : AnnData
        Filtered CD45+ data
    
    Returns:
    --------
    adata_processed : AnnData
        Processed CD45+ data with cell type annotations
    """
    print("Processing and clustering CD45+ immune cells...")
    
    # Save raw data
    adata_cd45_pos.raw = adata_cd45_pos
    
    # Normalize and log-transform
    sc.pp.normalize_total(adata_cd45_pos, target_sum=1e4)
    sc.pp.log1p(adata_cd45_pos)
    
    # Find highly variable genes
    sc.pp.highly_variable_genes(adata_cd45_pos, min_mean=0.0125, max_mean=3, min_disp=0.5)
    n_hvgs = adata_cd45_pos.var['highly_variable'].sum()
    print(f"Found {n_hvgs} highly variable genes in immune cells")
    
    # Scale and PCA
    sc.pp.scale(adata_cd45_pos, max_value=10)
    sc.tl.pca(adata_cd45_pos, svd_solver='arpack', n_comps=50)
    
    # Build neighborhood graph and UMAP
    sc.pp.neighbors(adata_cd45_pos, n_neighbors=10, n_pcs=40)
    sc.tl.umap(adata_cd45_pos)
    
    # Leiden clustering with higher resolution for immune cell subtypes
    sc.tl.leiden(adata_cd45_pos, resolution=0.04, key_added='leiden_0.04')
    sc.tl.leiden(adata_cd45_pos, resolution=1.0, key_added='leiden_1.0')
    sc.tl.leiden(adata_cd45_pos, resolution=1.5, key_added='leiden_1.5')
    sc.tl.leiden(adata_cd45_pos, resolution=2.0, key_added='leiden_2.0')
    
    # Use resolution 1.5 for immune cell clustering
    adata_cd45_pos.obs['leiden'] = adata_cd45_pos.obs['leiden_0.04']
    n_clusters = adata_cd45_pos.obs['leiden'].nunique()
    print(f"Identified {n_clusters} immune cell clusters")
    
    # Find marker genes for immune clusters
    sc.tl.rank_genes_groups(adata_cd45_pos, 'leiden', method='wilcoxon', use_raw=True)
    
    return adata_cd45_pos

def annotate_immune_cell_types(adata_cd45_pos):
    """
    Annotate immune cell clusters based on canonical markers
    
    Parameters:
    -----------
    adata_cd45_pos : AnnData
        Clustered CD45+ data
    
    Returns:
    --------
    adata_annotated : AnnData
        CD45+ data with immune cell type annotations
    """
    print("Annotating immune cell types based on canonical markers...")
    
    # Define immune cell markers
    immune_markers = {
        'T_cells': ['CD3E', 'CD3D', 'CD3G', 'IL7R'],
        'CD4_T_cells': ['CD4', 'IL7R', 'LTB'],
        'CD8_T_cells': ['CD8A', 'CD8B', 'GZMK'],
        'NK_cells': ['GNLY', 'NKG7', 'GZMA', 'KLRD1'],
        'B_cells': ['MS4A1', 'CD79A', 'CD79B', 'IGHM'],
        'Plasma_cells': ['IGHG1', 'MZB1', 'SDC1', 'XBP1'],
        'Macrophages': ['CD68', 'CD163', 'MSR1', 'MRC1'],
        'Monocytes': ['CD14', 'FCGR3A', 'LYZ', 'S100A8'],
        'Dendritic_cells': ['FCER1A', 'CD1C', 'CLEC9A', 'IRF8'],
        'Neutrophils': ['FCGR3B', 'CSF3R', 'FPR1', 'CXCR2'],
        'Mast_cells': ['TPSAB1', 'CPA3', 'MS4A2', 'HDC']
    }
    
    # Get marker genes and cluster assignments
    marker_genes = sc.get.rank_genes_groups_df(adata_cd45_pos, None)
    
    # Simple annotation based on top markers
    cluster_annotations = {}
    
    for cluster in sorted(adata_cd45_pos.obs['leiden'].unique()):
        cluster_markers = marker_genes[marker_genes['group'] == cluster].head(100)
        cluster_gene_set = set(cluster_markers['names'].tolist())
        
        best_match = "Unknown_immune"
        best_score = 0
        
        # Compare with immune cell markers
        for cell_type, markers in immune_markers.items():
            overlap = len(cluster_gene_set.intersection(set(markers)))
            if overlap > best_score:
                best_score = overlap
                best_match = cell_type
        
        cluster_annotations[cluster] = best_match if best_score > 0 else f"Immune_cluster_{cluster}"
    
    # Add annotations
    adata_cd45_pos.obs['cell_type'] = adata_cd45_pos.obs['leiden'].map(cluster_annotations)
    
    print("\nImmune cell type annotations:")
    for cluster, cell_type in cluster_annotations.items():
        n_cells = (adata_cd45_pos.obs['leiden'] == cluster).sum()
        print(f"Cluster {cluster}: {cell_type} ({n_cells} cells)")
    
    # Visualize immune cell clustering and annotation
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Clusters
    sc.pl.umap(adata_cd45_pos, color='leiden', legend_loc='on data', legend_fontsize=8,
               size=30, ax=axes[0], show=False, frameon=False)
    axes[0].set_title('CD45+ Immune Cell Clusters', fontsize=14, fontweight='bold')
    
    # Cell types
    sc.pl.umap(adata_cd45_pos, color='cell_type', legend_loc='right margin', legend_fontsize=8,
               size=30, ax=axes[1], show=False, frameon=False)
    axes[1].set_title('Immune Cell Type Annotation', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    plt.savefig('./data_and_results/scRNA_seq/GSE235449/figures/scrnaseq_cell_type_annotation.pdf', bbox_inches='tight')
    plt.close()

    # Print cell type composition
    print("\nImmune cell type composition:")
    cell_type_counts = adata_cd45_pos.obs['cell_type'].value_counts()
    for cell_type, count in cell_type_counts.items():
        percentage = (count / adata_cd45_pos.n_obs) * 100
        print(f"  {cell_type}: {count:,} cells ({percentage:.1f}%)")
    
    return adata_cd45_pos

# Process CD45+ dataset
print("="*80)
print("PROCESSING CD45-POSITIVE IMMUNE CELL DATASET (GSE235449)")
print("="*80)

# Define path to CD45+ data - modify this according to your file format
CD45_POS_RAW_PATH = "./data_and_results/scRNA_seq/GSE235449/adata_all_raw.h5ad"  # Adjust path as needed

# Alternative paths based on your data format:
# CD45_POS_RAW_PATH = "path/to/GSE235449/matrix.mtx.gz"  # For 10X mtx format
# CD45_POS_RAW_PATH = "GSE235449_expression_matrix.csv"  # For CSV format
# CD45_POS_RAW_PATH = "GSE235449_filtered_feature_bc_matrix.h5"  # For 10X h5 format

# If you need to download from GEO, uncomment below:
# import GEOparse
# gse = GEOparse.get_GEO("GSE235449")
# # Process GEO data as needed

# Step 1: Load and process CD45+ data
adata_cd45_pos = process_cd45_positive_data(CD45_POS_RAW_PATH)

# Step 2: QC and filtering
adata_cd45_pos = qc_and_filter_cd45_positive(adata_cd45_pos)

# Step 3: Processing and clustering
adata_cd45_pos = process_and_cluster_cd45_positive(adata_cd45_pos)

# Step 4: Immune cell type annotation
adata_cd45_pos = annotate_immune_cell_types(adata_cd45_pos)

# Save processed CD45+ data
adata_cd45_pos.write('./data_and_results/scRNA_seq/GSE235449/adata_processed_immune_cells.h5ad', compression='gzip')
print(f"Processed CD45+ data saved: {adata_cd45_pos.n_obs} cells × {adata_cd45_pos.n_vars} genes")

# Quick integration validation
print("\n" + "="*60)
print("INTEGRATION VALIDATION")
print("="*60)

# Visualize both datasets before integration
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('CD45- and CD45+ Datasets Before Integration', fontsize=16)

# Load CD45- data for comparison visualization
adata_cd45_neg_temp = sc.read_h5ad("./data_and_results/scRNA_seq/GSE194247/adata_all_preprocessed_annotated.h5ad")

# CD45- UMAP
sc.pl.umap(adata_cd45_neg_temp, color='cell_type', size=20, 
           ax=axes[0, 0], show=False, frameon=False)
axes[0, 0].set_title('CD45- Stromal Cell Types\n(Your 5 Major Types)')

# CD45+ UMAP
sc.pl.umap(adata_cd45_pos, color='cell_type', size=20,
           ax=axes[0, 1], show=False, frameon=False)
axes[0, 1].set_title('CD45+ Immune Cell Types\n(GSE235449)')

# Cell type counts comparison
cd45_neg_counts = adata_cd45_neg_temp.obs['cell_type'].value_counts()
cd45_pos_counts = adata_cd45_pos.obs['cell_type'].value_counts()

axes[1, 0].bar(range(len(cd45_neg_counts)), cd45_neg_counts.values, color='skyblue')
axes[1, 0].set_xticks(range(len(cd45_neg_counts)))
axes[1, 0].set_xticklabels(cd45_neg_counts.index, rotation=45, ha='right')
axes[1, 0].set_title('CD45- Cell Counts')
axes[1, 0].set_ylabel('Number of Cells')

axes[1, 1].bar(range(len(cd45_pos_counts)), cd45_pos_counts.values, color='lightcoral')
axes[1, 1].set_xticks(range(len(cd45_pos_counts)))
axes[1, 1].set_xticklabels(cd45_pos_counts.index, rotation=45, ha='right')
axes[1, 1].set_title('CD45+ Cell Counts')
axes[1, 1].set_ylabel('Number of Cells')

plt.tight_layout()
plt.show()
plt.savefig('./data_and_results/scRNA_seq/scrnaseq_cell_type_counts_comparison_cd45_pos_neg.pdf', bbox_inches='tight')
plt.close()
print(f"✓ CD45- dataset: {adata_cd45_neg_temp.n_obs:,} cells, {len(cd45_neg_counts)} cell types")
print(f"✓ CD45+ dataset: {adata_cd45_pos.n_obs:,} cells, {len(cd45_pos_counts)} cell types")
print(f"✓ Ready for integration and spatial deconvolution")

"""
INTEGRATION QUALITY ASSESSMENT:

DATASET BALANCE:
- Check that both CD45- and CD45+ populations are well-represented
- Ensure sufficient cells per cell type for robust signatures
- Validate cell type annotations make biological sense

EXPECTED PROPORTIONS:
- CD45- cells: typically 60-80% in most tissues
- CD45+ cells: typically 20-40% depending on inflammation status
- Balance affects deconvolution accuracy

CELL TYPE DIVERSITY:
- CD45-: 5 major stromal/parenchymal types
- CD45+: 6-10 immune cell types expected
- Comprehensive coverage enables accurate spatial mapping

The integration is now ready to proceed with spatial deconvolution!
"""

PROCESSING CD45-POSITIVE IMMUNE CELL DATASET (GSE235449)
Processing CD45-positive immune cell dataset (GSE235449)...
Loaded CD45+ data: 8297 cells × 33538 genes
Calculating QC metrics for immune cells...
CD45+ QC - Mean genes: 1667, Mean UMI: 6103, Mean MT%: 6.4%
Applying QC filters to CD45+ data...
Before filtering: 8297 cells
filtered out 625 cells that have less than 1000 counts
After UMI filter (>1000): 7672 cells
filtered out 76 cells that have less than 300 genes expressed
After gene filter (300-5000): 7533 cells
After MT filter (<15%): 7338 cells
filtered out 14639 genes that are detected in less than 3 cells
After gene filtering: 18899 genes
Final filtered CD45+ dataset: 7338 cells × 18899 genes
Processing and clustering CD45+ immune cells...
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (a

'\nINTEGRATION QUALITY ASSESSMENT:\n\nDATASET BALANCE:\n- Check that both CD45- and CD45+ populations are well-represented\n- Ensure sufficient cells per cell type for robust signatures\n- Validate cell type annotations make biological sense\n\nEXPECTED PROPORTIONS:\n- CD45- cells: typically 60-80% in most tissues\n- CD45+ cells: typically 20-40% depending on inflammation status\n- Balance affects deconvolution accuracy\n\nCELL TYPE DIVERSITY:\n- CD45-: 5 major stromal/parenchymal types\n- CD45+: 6-10 immune cell types expected\n- Comprehensive coverage enables accurate spatial mapping\n\nThe integration is now ready to proceed with spatial deconvolution!\n'